Здесь неудавшаяся модель на fbprophet, не успел ее довести до ума

In [ ]:
# группируем минутки в данные h1 и добавляем волатильность 
ohlc_dict = {'Open':'first', 'High':'max', 'Low':'min', 'Close': 'last'}
h1_quotes = m1_quotes.resample('H').apply(ohlc_dict)
#h1_quotes = h1_quotes[h1_quotes.index.weekday < 5] # удаляем СБ и ВСК с нулевой волатильностью по итогам 1 этапа
# волатильность - амплитуду движения за день - считаем в пунктах по 5-му знаку,
# умножая для этого на 10^5
h1_quotes['volatility'] = (abs(h1_quotes.High - h1_quotes.Low) * 10**5).astype(int)
h1_quotes = h1_quotes.drop(['Open', 'High', 'Low', 'Close'], axis=1)
h1_quotes[100:].head()

In [ ]:
# делим данные в соотношение 3/4

df = sparse_news_h1.reset_index()
df = df.rename(columns={'date': 'ds', 'volatility': 'y'})

df_train = df[:int(rows_num*.75)]
df_test = df[int(rows_num*.75):]

# на столько будем прогнозировать
predictions = (df_test['ds'].iloc[-1] - df_test['ds'].iloc[0]).days

#y_train = df_train.volatility
#y_test = df_test.volatility
#X_train = df_train.drop('volatility', axis=1)
#X_test = df_test.drop('volatility', axis=1)

### Обучение модели

Используем библиотеку FB для работы с временными рядами - fbprophet. Бибилотека умеет работать с временными данными без предварительной подготовки, выделяя самостоятельно сезонные зависимости, но в учебных целях выделение и кодирование часов и дня недели выше выполнено.

In [ ]:
from fbprophet import Prophet
model = Prophet()
model.fit(df_train)

При обучении модели автоматически отключилась годовая сезонность, что ожидаемо, т.к. рассматриваются данные внутри одного года

In [ ]:
# С помощью вспомогательной функции Prophet.make_future_dataframe создаем dataframe,
# который содержит все исторические временные точки и еще predictions дней, для которых планируем построить прогноз
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
forecast.head(3)

In [ ]:
model.plot(forecast);

In [ ]:
model.plot_components(forecast);

In [ ]:
forecast.set_index('ds')[['yhat']].join(df_test[['y', 'ds']].set_index('ds'))